In [35]:
#Importing libraries

import json
import csv
import matplotlib
import numpy as np

In [84]:
# The order of La liga at the end of the season
#The idea is to use this to predict the result of the matches, ie Barcelona won all of them and levante won none
order ={}
order[1] = 'barcelona'
order[2] = 'real madrid'
order[3] = 'ath madrid'
order[4] = 'villarreal'
order[5] = 'ath bilbao'
order[6] = 'celta vigo'
order[7] = 'sevilla'
order[8] = 'malaga'
order[9] = 'sociedad'
order[10] = 'real betis'
order[11] = 'las palmas'
order[12] = 'valencia'
order[13] = 'espanyol'
order[14] = 'eibar'
order[15] = 'deportivo'
order[16] = 'granada'
order[17] = 'sporting'
order[18] = 'rayo'
order[19] = 'getafe'
order[20] = 'levante'

#getting the position of the team
def get_position(team):
    min_lev = [30, 99] #position, distance lev
    for key,t in order.iteritems():
        dist = lev(team,t)
        if dist<= min_lev[1]:
            min_lev[0] = key
            min_lev[0] = dist
    return min_lev[0]
    

In [85]:
#this is necessary because of the accents an other errors from scrapping around
def levenshtein(s1, s2):
    #https://en.wikipedia.org/wiki/Levenshtein_distance
    if len(s1) < len(s2):
        return lev(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def lev(s1,s2):
    #convert to lower everything to avoid mistakes
    return levenshtein(s1.lower(),s2.lower())

In [97]:
#Reading the bets and other info
bets = np.genfromtxt('/home/roc/workspace/fifa-db-players/match-results/1516.csv', delimiter=',', skip_header=1
                                ,dtype="|S10")

#matches
matches = bets[:,2:4]
aux = bets[:,6]
results = []
for r in aux:
    if r == 'D':
        results.append(0)
    elif r == 'A':
        results.append(1)
    else: #H
        results.append(-1)


In [102]:
#Result match my encoding
#homies wins -1
#tie 0
#outer wins 1

my_results_basic = []
for match in matches:
    p1 = get_position(match[0])
    p2 = get_position(match[1])

    #with only win/lose it gets 1% improvement
    if p1+1 == p2 or p2+1==p1:
        my_results_basic.append(0)
    elif p1>p2:
        my_results_basic.append(-1)
    else:
        my_results_basic.append(1)

    
count = 0.0
for i in range(len(results)):
    if results[i] == my_results_basic[i]:
        count +=1
print "based on the final Liga order accuracy", count/len(results)*100.0,'%'

based on the final Liga order accuracy 46.3157894737 %
